<a href="https://colab.research.google.com/github/mattijn/datasets/blob/master/WaterData_Scheldestromen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installatie + importeren Python packages in de Colab Notebook

Allereerst gaan we drie Python packages installeren welke niet standaard binnen de Colab omgeving zijn geïnstalleerd. Deze gaan we installeren via het package management system **`pip`**.  Dit roepen we aan binnen de Notebook als een *magic* functie, namelijk via

```python
!pip install <package_name>
```


---



De drie Python packages welke we gaan installeren zijn `hkvfewspy`, `hkvportal` en `geopandas`. 

In [1]:
!pip install hkvfewspy
!pip install hkvportal
!pip install geopandas

     |████████████████████████████████| 102kB 7.2MB/s 
     |████████████████████████████████| 51kB 20.8MB/s 
     |████████████████████████████████| 61kB 24.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
Successfully built fire
     |████████████████████████████████| 901kB 2.8MB/s 
     |████████████████████████████████| 17.9MB 36.6MB/s 
     |████████████████████████████████| 11.2MB 37.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


Vervolgens gaan we een aantal Python packages importeren binnen onze notebook. Elk van deze packages bevat een bibliotheek van functies welke we gaan gebruiken in deze workshop.


---



- `hkvfewspy` - voor het uitwisselen van gegevens met een FEWS database
- `pandas` - voor het werken met data welke maximaal 2-dimensies heeft (net zoals Microsoft Excel dat ook kan).
- `altair` - voor het maken van interactieve visualisaties
- `StringIO` - voor een in-memory text stream van HTML paginas
- `hkvportal` - voor het beschikbaar maken van de visualisatie in de browser (via https)

In [0]:
import hkvfewspy
import pandas
import altair
from io import StringIO
from hkvportal.io.services import dataportal as hkvportal

### Verbinding maken naar de FEWS Database + Dataportal 

Nu we de juiste Python packages hebben geinstalleerd en geimporteerd kunnen we beginnen met het opzetten van een verbinding naar de FEWS WebService welke beschikbaar is gesteld voor deze workshop. 

Dit doen we door middel van de FEWS WebService, wat een soort van API is waarmee we op een gestandardiseerde manier kunnen praten met de database van FEWS.

We doen dit door een `pi` object te initieren, waarbij we een `setClient` functie uitvoeren. In deze functie plaatsen we de URL van de FEWS PiService.

Daarnaast maken we ook een connectie naar de HKV dataportal welke we gebruiken voor het 'serveren' van de nog aan te maken visualisaties. 

In [3]:
# wordt gebruikt voor toegang FewsPi service Scheldestromen
pi = hkvfewspy.Pi() 
pi.setClient(wsdl='https://webviewer.hkvfewsscheldestromen.nl:8443/FewsPiService/fewspiservice?wsdl')

# wordt gebruikt voor opslag visualisaties voor Scheldestromen Sharepoint
dp = hkvportal()
dp.setDataservice(dataservice='https://dmws.hkvservices.nl/dataportal/')

dataservice is set. https://dmws.hkvservices.nl/dataportal/ will be used as portal


### Opvragen Locaties en Parameters

Met de opgezette verbinding gaan we de locaties opvragen zoals deze bekend zijn in de FEWS database. Dit doen we met de functie `getLocations()`. Deze locaties slaan we op in de variabele `locations`. We tonen deze `locations` vervolgens als een `pandas.DataFrame`.

In [4]:
# get available locations
locations = pi.getLocations()
locations

,VOV9345,VOV9346,VOV32379,KGM8,KGM44,KGM34,KGM54,KGM12,KGM39,KGM100,KGM46,KGM156,KGM157,KGM73,KGM55,KGM36,KGM35,KGM51,KGM19,KGM20,KGM159,KGM151,KGM131,KGM135,KGM142,KGM52,KGM10,KGM91,KGM27,KGM182,KGM17,KGM65,KGM97,KGM21,KGM49,KGM57,KGM90,KGM47,KGM9,KGM76,...,MPN4462,MPN4458,MPN4461,MPN4459,MPN4466,MPN4457,MPN4465,MPN4463,MPN4473,MPN4474,MPN4480,MPN4479,MPN4475,MPN4478,MPN4481,MPN4482,MPN4483,MPN4484,MPN4485,MPN4486,MPN4487,MPN4488,MPN4489,MPN4494,MPN4493,MPN4492,MPN4498,MPN4499,MPN4496,MPN4497,MPN4904,MPN4905,MPN4906,MPN4909,MPN4907,MPN4913,MPN4912,MPN4911,MPN4910,MPN4914
geoDatum,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,...,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel,Rijks Driehoekstelsel
lat,51.4016519479478,51.48123119476028,51.27317529478825,51.65463775239045,51.563714349508196,51.507909256439255,51.38599635618589,51.64885045386372,51.545244272349386,51.572925598764755,51.55620270269095,51.33878080346948,51.30605173757065,51.60505885439964,51.406660036967075,51.45864422742382,51.49180954345002,51.412049562563794,51.59100966484886,51.59059882163103,51.33577866963928,51.29832669945125,51.3769428510576,51.36442677768085,51.29319882882253,51.38727719968617,51.7305472875512,51.50985984073714,51.52587544998866,51.404660714541336,51.613908249533615,51.480795124070845,51.5534462646436,51.55345650109916,51.518557399723036,51.47500391106764,51.420621177963035,51.601469444527865,51.74072824485289,51.57157406003172,...,51.28188177817269,51.277086446137695,51.30598166372969,51.248498025292825,51.29925893558335,51.31657031172857,51.30522379258914,51.36051886008018,51.58752477809138,51.64427029590638,51.707514214323076,51.70616609672372,51.71108649816933,51.67317260931914,51.73421938588256,51.73421381296282,51.305789411502296,51.39751341342654,51.485452770448305,51.44111887356126,51.45190469796753,51.482012658977446,51.483148787912754,51.44442421528728,51.44441715380222,51.483080472644275,51.44412293030843,51.39057776831087,51.417529899110754,51.4175177407747,51.46058131867781,51.46058979422694,51.49240556082831,51.47775839475506,51.4777314717713,51.53571173699996,51.50091659992924,51.51109798969467,51.51919839664145,51.52437906490621
locationId,VOV9345,VOV9346,VOV32379,KGM8,KGM44,KGM34,KGM54,KGM12,KGM39,KGM100,KGM46,KGM156,KGM157,KGM73,KGM55,KGM36,KGM35,KGM51,KGM19,KGM20,KGM159,KGM151,KGM131,KGM135,KGM142,KGM52,KGM1

Wat we zien is een dik gedrukte eerste rij (*VOV9345, VOV9346, VOV32379, ..*). Dit zijn de `columns` van de `DataFrame`. Daarnaast zien we een dik gedrukte eerste kolom (*geoDatum, lat, locationId, lon, shortName, x, y*), dit is de `index` van de `DataFrame`.

---



Met behulp van de `columns` en `index`waarden kunnen we een bepaalde rij/kolom of cel selecteren, bijvoorbeeld de **`x`** waarde welke hoort bij locatie **`KST457`**.

In [5]:
# slice by column and row
locations.KST457.x

'41666.77'

In [0]:
# wat is de shortName voor ID KGM157? (answer: locations.KGM157.shortName)

Hetzelfde gaan we doen voor de mogelijke parameters. Dit doen we door de functie `getParameters` uit te voeren, waarbij we de resultaten schrijven naar de variabele `parameters`. Net als `locations` is het resultaat hiervan een `pandas.DataFrame`.

In [7]:
# get available parameters
parameters = pi.getParameters()
parameters

,WNSHDB1,WNSHDB3,WNSHDB5,WNSHDB6,WNSHDB7,WNSHDB8,WNSHDB27,WNSHDB38,WNSHDB96
displayUnit,mNAP,mNAP,-,-,-,-,mS/cm,mNAP,-
id,WNSHDB1,WNSHDB3,WNSHDB5,WNSHDB6,WNSHDB7,WNSHDB8,WNSHDB27,WNSHDB38,WNSHDB96
name,waterstand instroom,waterstand uitstroom,bedrijf pomp1,bedrijf pomp2,bedrijf pomp3,bedrijf pomp4,geleidendheid,waterstand peilschaal,bedrijf pomp5
parameterType,instantaneous,instantaneous,instantaneous,instantaneous,instantaneous,instantaneous,instantaneous,instantaneous,instantaneous
unit,mNAP,mNAP,-,-,-,-,mS/cm,mNAP,-
usesDatum,false,false,false,false,false,false,false,false,false


Zoals we zien zijn de beschikbare parameters bekend onder de namen *WNSHDB1*,	*WNSHDB3*,	*..* en	*WNSHDB96*. Daarnaast is er ook wat meer informatie beschikbaar over deze parameters. De zogenoemde metadata.

Wanneer we bijvoorbeeld de metadata willen weten van parameter `WNSHDB1` doen we het volgende:

In [8]:
# slice by column
parameters.WNSHDB1

displayUnit                     mNAP
id                           WNSHDB1
name             waterstand instroom
parameterType          instantaneous
unit                            mNAP
usesDatum                      false
Name: WNSHDB1, dtype: object

In [0]:
# wat is de unit van parameter WNSHDB27? (answer: parameters.WNSHDB27.unit)

### Opvragen Tijdsreeks vanuit de FEWS database

Vervolgens willen we met behulp van deze bekende locaties en parameters een tijdreeks uitlezen uit de FEWS database. Hiervoor gaan we een `query` object aan maken met de functie `setQueryParameters`. 

Dit is een hulp object waarbij we specifiek onze opties opgeven zodat de FEWS database in staat is om de juiste tijdreeks terug te geven.

In [0]:
# initiate query object
query = pi.setQueryParameters()

Binnen de `query` zijn we geinteresseerd in:
- de `parameterIds` welke horen bij de instromende waterstand (`WNSHDB1`) en de uitstromende waterstand (`WNSHDB3`)
- de `locationIds` van de stuwen `KST457`, `KST291` en `KST521`.
- voor de periode tussen 3 dagen geleden (`startTime`) en nu (`endTime`).

In [0]:
eind = pandas.datetime.now()
start = eind - pandas.Timedelta(3, 'days')

In [12]:
start

datetime.datetime(2019, 6, 9, 13, 20, 4, 718376)

In [0]:
# define query parameters
query.parameterIds(['WNSHDB1', 'WNSHDB3'])
query.locationIds(['KST457', 'KST291', 'KST521'])
query.startTime(start)
query.endTime(eind)

In [14]:
query.query

{'clientTimeZone': 'Etc/GMT',
 'convertDatum': False,
 'endTime': datetime.datetime(2019, 6, 12, 13, 20, 4, 718376),
 'forecastSearchCount': 0,
 'importFromExternalDataSource': False,
 'locationIds': ['KST457', 'KST291', 'KST521'],
 'omitMissing': False,
 'onlyHeaders': False,
 'onlyManualEdits': False,
 'parameterIds': ['WNSHDB1', 'WNSHDB3'],
 'showEnsembleMemberIds': False,
 'showLocationAttributes': False,
 'showStatistics': False,
 'showThresholds': False,
 'startTime': datetime.datetime(2019, 6, 9, 13, 20, 4, 718376),
 'useDisplayUnits': True,
 'version': '1.25'}

Met het aangemaakte `query` object vragen we de tijdreeks op met behulp van de `getTimeSeries` functie, waar we ons `query` object meegeven waarbij we het resultaat plaatsen in de variabele `timeseries`.

Het resultaat is weer een pandas.DataFrame. Sinds we voor 3 locaties 2 parameters opvragen voor een periode van 3 dagen met kwartier waarden willen we niet alles tonen, maar alleen de bovenste 5 regels. Dit doen we met de regel:

```python
timeseries.head()
```

*Note: Het opvragen van deze tijdreeks combinatie kan een poosje duren. *

In [15]:
# request timeseries from FEWS given the query parameters
timeseries = pi.getTimeSeries(query)
timeseries.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KST291,KST291 - Stuw Steursweg,0,-1.886
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",0,-1.112
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KST521,KST521 - Stuw Ridderweg,0,-2.587
2019-06-09 13:15:00+00:00,,,WNSHDB3,mNAP,KST291,KST291 - Stuw Steursweg,0,-2.605
2019-06-09 13:15:00+00:00,,,WNSHDB3,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",0,-1.858


In [16]:
timeseries.tail()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-06-12 13:30:00+00:00,,,WNSHDB1,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",8,-999
2019-06-12 13:30:00+00:00,,,WNSHDB1,mNAP,KST521,KST521 - Stuw Ridderweg,8,-999
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST291,KST291 - Stuw Steursweg,8,-999
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",8,-999
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST521,KST521 - Stuw Ridderweg,8,-999


In [0]:
# hoeveel waarden hebben we nu opgevraagd? (gebruik eg. timeseries.shape)

Wanneer we de DataFrame analyseren vallen er een aantal dingen op. Namelijk een `index` met de tijden en een aantal `columns` met FEWS eigenschappen van de tijdreeks. Met vervolgens de kolom `values` waarin de daadwerkelijke getallen staan die gemeten en/of afgeleid zijn.

In de `flag` kolom zien we de waarden 8 en 0 terug komen. Deze waarden geven een bepaalde betekenis over de waarde in de kolom `value`.

Deze flags hebben de volgende betekenis volgens de FEWS documentatie¹:

---



| Enumeration | Description                                                                                                                                        |
|-------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| 0           | Original/Reliable   The data value is the original value retrieved from an external source and it successfully passes all validation criteria set. |
| 8           | Completed/Unreliable   Original value was missing. Value has been filled in as above, but resulting value is unreliable and is removed.            |


---


Zoals te zien zijn de waarden met een `flag` 8 missing values. Deze willen we dan ook maskeren in de DataFrame

*¹ Bron: https://publicwiki.deltares.nl/display/FEWSDOC/D+Time+Series+Flags*


In [18]:
# mask nodata values using the flag value
timeseries.value.mask(timeseries.flag == 8, inplace=True)
timeseries.tail()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-06-12 13:30:00+00:00,,,WNSHDB1,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",8,NaN
2019-06-12 13:30:00+00:00,,,WNSHDB1,mNAP,KST521,KST521 - Stuw Ridderweg,8,NaN
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST291,KST291 - Stuw Steursweg,8,NaN
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST457,"KST457 - Stuw Meeldijk, nr 2",8,NaN
2019-06-12 13:30:00+00:00,,,WNSHDB3,mNAP,KST521,KST521 - Stuw Ridderweg,8,NaN


### Interactieve visualisatie Tijdreeks van de waterstanden

Met de DataFrame waarbij de missing values gemaskeerd zijn (als `NaN`) willen we een interactieve visualisatie maken waarbij:
- de `x`-as de tijd toont
- de `y`-as de gemeten waarden
- de 3 locaties zichtbaar zijn als gekleurde lijnen (`color`)
- de 3 locaties naast elkaar worden geplaatst in een eigen grafiek (`column`-facet)
- de 2 parameters (*WNSHDB1* en *WNSHDB3*) onder elkaar worden geplaatst in een eigen grafiek (`row`-facet).

Als interactie willen we:
- kunnen inzoomen + pannen op de `x`-as 
- de *value*, *stationName* en *date*  zien oplichten als we over de punten gaan (`tooltip`)


In [19]:
# setup an interactive line chart
line_chart = altair.Chart(timeseries.reset_index()).mark_line(point=True).encode(
    x='date',
    y=altair.Y('value', title='waterstand (mNAP)',scale=altair.Scale(zero=False)),
    color='stationName',
    tooltip=['value', 'stationName', altair.Tooltip('date', format='%Y-%m-%d %H:%M')],
    column='stationName',
    row='parameterId'       
).properties(
    width=240,
    height=150
).resolve_scale(
    y='independent'
).interactive(bind_y=False)

# diplay the chart
line_chart

Chart({
  data:                           date moduleInstanceId  ... flag  value
  0    2019-06-09 13:15:00+00:00                   ...    0 -1.886
  1    2019-06-09 13:15:00+00:00                   ...    0 -1.112
  2    2019-06-09 13:15:00+00:00                   ...    0 -2.587
  3    2019-06-09 13:15:00+00:00                   ...    0 -2.605
  4    2019-06-09 13:15:00+00:00                   ...    0 -1.858
  5    2019-06-09 13:15:00+00:00                   ...    0 -2.588
  6    2019-06-09 13:30:00+00:00                   ...    0  -1.89
  7    2019-06-09 13:30:00+00:00                   ...    0 -1.111
  8    2019-06-09 13:30:00+00:00                   ...    0 -2.586
  9    2019-06-09 13:30:00+00:00                   ...    0 -2.602
  10   2019-06-09 13:30:00+00:00                   ...    0  -1.86
  11   2019-06-09 13:30:00+00:00                   ...    0 -2.587
  12   2019-06-09 13:45:00+00:00                   ...    0 -1.889
  13   2019-06-09 13:45:00+00:00                   ...    0  -1.11
  14   2019-06-09 13:45:00+00:00                   ...    0 -2.586
  15   2019-06-09 13:45:00+00:00                   ...    0 -2.599
  16   2019-06-09 13:45:00+00:00                   ...    0  -1.86
  17   2019-06-09 13:45:00+00:00                   ...    0 -2.587
  18   2019-06-09 14:00:00+00:00                   ...    0 -1.888
  19   2019-06-09 14:00:00+00:00                   ...    0 -1.109
  20   2019-06-09 14:00:00+00:00                   ...    0 -2.585
  21   2019-06-09 14:00:00+00:00                   ...    0 -2.596
  22   2019-06-09 14:00:00+00:00                   ...    0 -1.859
  23   2019-06-09 14:00:00+00:00                   ...    0 -2.587
  24   2019-06-09 14:15:00+00:00                   ...    0 -1.888
  25   2019-06-09 14:15:00+00:00                   ...    0 -1.108
  26   2019-06-09 14:15:00+00:00                   ...    0 -2.585
  27   2019-06-09 14:15:00+00:00                   ...    0 -2.593
  28   2019-06-09 14:15:00+00:00                   ...    0 -1.859
  29   2019-06-09 14:15:00+00:00                   ...    0 -2.586
  ...                        ...              ...  ...  ...    ...
  1710 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1711 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1712 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1713 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1714 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1715 2019-06-12 12:30:00+00:00                   ...    8    NaN
  1716 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1717 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1718 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1719 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1720 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1721 2019-06-12 12:45:00+00:00                   ...    8    NaN
  1722 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1723 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1724 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1725 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1726 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1727 2019-06-12 13:00:00+00:00                   ...    8    NaN
  1728 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1729 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1730 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1731 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1732 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1733 2019-06-12 13:15:00+00:00                   ...    8    NaN
  1734 2019-06-12 13:30:00+00:00                   ...    8    NaN
  1735 2019-06-12 13:30:00+00:00                   ...    8    NaN
  1736 2019-06-12 13:30:00+00:00                   ...    8    NaN
  1737 2019-06-12 13:30:00+00:00 

### Export naar URL (secured)

Dit is een figuur welke we willen tonen in de Documenten Atelier oftewel SharePoint online. 

We doen dit door de visualisatie om te zetten naar een HTML pagina. Deze HTML pagina kunnen we vervolgens insluiten binnen een door ons opgemaakte SharePoint pagina.

Om het figuur om te zetten naar een HTML pagina, kunnen we deze naar een `html` formaat opslaan. Normaal gesproken schrijf je dit naar een bestand op je harde schijf. Maar nu willen we deze figuur wegschrijven naar een secured (https) internet-pagina. Hiervoor gebruiken we in deze training de package `hkvportal`.

We gaan het figuur dus eerst omzetten naar een HTML-pagina representatie als tekst.


In [0]:
# export visualizatie als een 'in-memory text stream'
export_html = StringIO()
line_chart.save(export_html, format='html')

In [21]:
# inspecteer de output van de in-memory text stream. Wat herken je?
export_html.getvalue()

'<!DOCTYPE html>\n<html>\n<head>\n  <style>\n    .vega-actions a {\n        margin-right: 12px;\n        color: #757575;\n        font-weight: normal;\n        font-size: 13px;\n    }\n    .error {\n        color: red;\n    }\n  </style>\n  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega@5"></script>\n  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-lite@3.3.0"></script>\n  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-embed@4"></script>\n</head>\n<body>\n  <div id="vis"></div>\n  <script>\n      var spec = {"config": {"view": {"width": 400, "height": 300}, "mark": {"tooltip": null}}, "data": {"name": "data-a71a2a358f8cb38af6bfe019ce2da796"}, "mark": {"type": "line", "point": true}, "encoding": {"color": {"type": "nominal", "field": "stationName"}, "column": {"type": "nominal", "field": "stationName"}, "row": {"type": "nominal", "field": "parameterId"}, "tooltip": [{"type": "quantitative", "field": "value"}, {"type

Nu we de HTML-pagina als in-memory text stream hebben hebben we de mogelijkheid om deze weg te schrijven naar de `hkvportal`. Dit doen we door de `database` aan te geven waarin we de HTML pagina willen opslaan en de `key` onder welke naam dit gedaan moet worden.

Gebruik voor `key` een eigen bedachte naam anders overschrijf je de figuren van elkaar (vervelend als ze anders zijn). 

Het resultaat is een URL waaronder het figuur beschikbaar komt. 

In [22]:
dp.setEntryDatabase(database='WaterDataScheldestromen', key='stuwen_series', 
                    data=export_html.getvalue(), 
                    description='waterstand tijdreeks van 3 stuwen')

available at https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=stuwen_series&contentType=SET_BY_USER


{'date': '2019-06-12 13:20:56',
 'id': '943901bd-5b76-4ca1-b8de-21a017d11452',
 'key': 'stuwen_series'}

Klik op de URL en inspecteer het resultaat.

Wat gebeurd er als je `contentType=SET_BY_USER` wijzigt in `contentType=text/plain`?
En wat als je `contentType=text/html` doet?

Huh? Hoe kan dit?!

In [0]:
# dp.setEntryDatabase(database='WaterDataScheldestromen', key='stuwen_data_json_series', 
#                     data=timeseries.to_json(orient='records'), 
#                     description='waterstand tijdreeks van 3 stuwen')

### Welke informatie uit FEWS wil je ophalen? En op welke manier zou je deze willen visualiseren? 

Probeer het maar eens!

### Visualisatie Geavanceerd.

Stel dat we bijvoorbeeld een visualisatie willen laten zien waarbij we alle sluizen van ons watersysteem willen tonen op een kaart door middel van stippen. En elke stip willen we varieren in grote op basis van de gemiddelde waterstand van de sluis locatie over de periode waarover we de data opvragen.

Daarnaast willen we ook nog dat je kan klikken op elke punt in de kaart en dat er vervolgens een tijdreeks van desbetreffend locatie wordt getoond.
En als we dan toch bezig zijn willen we ook nog meerdere sluizen kan selecteren zodat ik de waterstanden met elkaar kan vergelijken.

En om een beetje duiding te geven gebruiken we de gebieden zoals gedefinieerd in de Plan WaterOpgave (`pwo_gebieden`). 
Uiteaard alles inzichtelijk met tooltips.



Dat betekent dat we eerst alle sluizen moeten filteren. Dit zijn locaties waarbij de `locationId` beginnen allen met `KSL`.

In [25]:
selected_locations = locations.T[locations.T.locationId.str.contains('KSL')]
selected_locations

,geoDatum,lat,locationId,lon,shortName,x,y
KSL4,Rijks Driehoekstelsel,51.440325603382384,KSL4,3.5757695823502744,KSL4 - Keersluis Koopmanshaven,29064.09,385034.19
KSL2,Rijks Driehoekstelsel,51.64228297649024,KSL2,3.910717194727634,KSL2 - Keersluis Zierikzee,52803.51,406974.0
KSL1,Rijks Driehoekstelsel,51.399340644272876,KSL1,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24
KSL5,Rijks Driehoekstelsel,51.38617630660279,KSL5,3.8172123801857114,KSL5 - Uitwateringssluis Hellewoud,45718.25,378621.68
KSL12,Rijks Driehoekstelsel,51.365630009138464,KSL12,3.691328279197892,KSL12 - Uitwateringssluis Nol Zeven,36903.77,376532.52
KSL14,Rijks Driehoekstelsel,51.382396631419375,KSL14,4.032760577860117,KSL14 - Uitwateringssluis Walsoorden,60712.76,377900.41
KSL8,Rijks Driehoekstelsel,51.442117205348524,KSL8,3.6227898273054473,KSL8 - Uitwateringssluis Zuidwatering,32337.45,385153.29


 Voor de Pi-Query moeten we de sluiz locaties als lijst hebben met enkel de Ids:

In [26]:
sel_locids = selected_locations.locationId.tolist()
sel_locids

['KSL4', 'KSL2', 'KSL1', 'KSL5', 'KSL12', 'KSL14', 'KSL8']

In [0]:
start = pandas.datetime.now()
eind = start + pandas.Timedelta(3, 'days')

In [28]:
# initiate query object
query = pi.setQueryParameters()

# define query parameters
query.parameterIds(['WNSHDB1'])
query.locationIds(sel_locids)
query.startTime(start)
query.endTime(eind)

# request timeseries from FEWS given the query parameters
timeseries_2 = pi.getTimeSeries(query)
timeseries_2.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-06-12 13:15:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,8,-999
2019-06-12 13:15:00+00:00,,,WNSHDB1,mNAP,KSL12,KSL12 - Uitwateringssluis Nol Zeven,8,-999
2019-06-12 13:15:00+00:00,,,WNSHDB1,mNAP,KSL14,KSL14 - Uitwateringssluis Walsoorden,8,-999
2019-06-12 13:15:00+00:00,,,WNSHDB1,mNAP,KSL2,KSL2 - Keersluis Zierikzee,8,-999
2019-06-12 13:15:00+00:00,,,WNSHDB1,mNAP,KSL4,KSL4 - Keersluis Koopmanshaven,8,-999


Moeten we nog `flag 8` waarden filteren? Controleer de tijd, gebruik de `tail()` functie. Klopt dit wel? Aaargh, we doen iets verkeerds! Maar wat?

In [30]:
eind = pandas.datetime.now()
start = eind - pandas.Timedelta(3, 'days')

# initiate query object
query = pi.setQueryParameters()

# define query parameters
query.parameterIds(['WNSHDB1'])
query.locationIds(sel_locids)
query.startTime(start)
query.endTime(eind)

# request timeseries from FEWS given the query parameters
timeseries_2 = pi.getTimeSeries(query)

# mask nodata values using the flag value
timeseries_2.value.mask(timeseries_2.flag == 8, inplace=True)

timeseries_2.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value
date,,,,,,,,
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.947
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL12,KSL12 - Uitwateringssluis Nol Zeven,0,0.01
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL14,KSL14 - Uitwateringssluis Walsoorden,0,-0.813
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL2,KSL2 - Keersluis Zierikzee,0,-1.264
2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL4,KSL4 - Keersluis Koopmanshaven,0,-1.164


Pffiew! Gelukkig heb je het kunnen oplossen. Laten we de tijdreeks koppelen aan de latitude en longitude welke we via de `getLocations()` functie hebben binnen gehaald.

Dit doen we door middel van een inner merge. Ook wel bekend onder de Excelse naamgeving VLOOKUP .

In [31]:
ts_latlon = timeseries_2.reset_index().merge(selected_locations, on='locationId')
ts_latlon.head()

,date,moduleInstanceId,qualifierId,parameterId,units,locationId,stationName,flag,value,geoDatum,lat,lon,shortName,x,y
0,2019-06-09 13:15:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.947,Rijks Driehoekstelsel,51.399340644272876,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24
1,2019-06-09 13:30:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.946,Rijks Driehoekstelsel,51.399340644272876,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24
2,2019-06-09 13:45:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.946,Rijks Driehoekstelsel,51.399340644272876,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24
3,2019-06-09 14:00:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.946,Rijks Driehoekstelsel,51.399340644272876,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24
4,2019-06-09 14:15:00+00:00,,,WNSHDB1,mNAP,KSL1,KSL1 - Uitwateringssluis Bath,0,-1.945,Rijks Driehoekstelsel,51.399340644272876,4.218835820498346,KSL1 - Uitwateringssluis Bath,73695.47,379561.24


Zoals je ziet hebben we nu voor elke waarde ook een locatie gekoppeld. Laten we deze gebruiken ter visualizatie. We gebruiken hier de `longitude` voor `lon`, `latitude` voor `lat`. En we wilden een bolletje hebben waarbij de grote van het bolletje afhangt van de gemiddelde waterstand welke over deze periode per sluis bekend is.

Daarnaast willen we een tooltip hebben welke aangeeft welke sluis het is.

In [32]:
# allereerst een ruimtelijk beeld waarbij de grote van het bolletje overeenkomt met de gemiddelde waterstand
altair.Chart(ts_latlon).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size='mean(value)',
    tooltip=['stationName']
)

Chart({
  data:                           date moduleInstanceId  ...         x          y
  0    2019-06-09 13:15:00+00:00                   ...  73695.47  379561.24
  1    2019-06-09 13:30:00+00:00                   ...  73695.47  379561.24
  2    2019-06-09 13:45:00+00:00                   ...  73695.47  379561.24
  3    2019-06-09 14:00:00+00:00                   ...  73695.47  379561.24
  4    2019-06-09 14:15:00+00:00                   ...  73695.47  379561.24
  5    2019-06-09 14:30:00+00:00                   ...  73695.47  379561.24
  6    2019-06-09 14:45:00+00:00                   ...  73695.47  379561.24
  7    2019-06-09 15:00:00+00:00                   ...  73695.47  379561.24
  8    2019-06-09 15:15:00+00:00                   ...  73695.47  379561.24
  9    2019-06-09 15:30:00+00:00                   ...  73695.47  379561.24
  10   2019-06-09 15:45:00+00:00                   ...  73695.47  379561.24
  11   2019-06-09 16:00:00+00:00                   ...  73695.47  379561.24
  12   2019-06-09 16:15:00+00:00                   ...  73695.47  379561.24
  13   2019-06-09 16:30:00+00:00                   ...  73695.47  379561.24
  14   2019-06-09 16:45:00+00:00                   ...  73695.47  379561.24
  15   2019-06-09 17:00:00+00:00                   ...  73695.47  379561.24
  16   2019-06-09 17:15:00+00:00                   ...  73695.47  379561.24
  17   2019-06-09 17:30:00+00:00                   ...  73695.47  379561.24
  18   2019-06-09 17:45:00+00:00                   ...  73695.47  379561.24
  19   2019-06-09 18:00:00+00:00                   ...  73695.47  379561.24
  20   2019-06-09 18:15:00+00:00                   ...  73695.47  379561.24
  21   2019-06-09 18:30:00+00:00                   ...  73695.47  379561.24
  22   2019-06-09 18:45:00+00:00                   ...  73695.47  379561.24
  23   2019-06-09 19:00:00+00:00                   ...  73695.47  379561.24
  24   2019-06-09 19:15:00+00:00                   ...  73695.47  379561.24
  25   2019-06-09 19:30:00+00:00                   ...  73695.47  379561.24
  26   2019-06-09 19:45:00+00:00                   ...  73695.47  379561.24
  27   2019-06-09 20:00:00+00:00                   ...  73695.47  379561.24
  28   2019-06-09 20:15:00+00:00                   ...  73695.47  379561.24
  29   2019-06-09 20:30:00+00:00                   ...  73695.47  379561.24
  ...                        ...              ...  ...       ...        ...
  2000 2019-06-12 06:15:00+00:00                   ...  32337.45  385153.29
  2001 2019-06-12 06:30:00+00:00                   ...  32337.45  385153.29
  2002 2019-06-12 06:45:00+00:00                   ...  32337.45  385153.29
  2003 2019-06-12 07:00:00+00:00                   ...  32337.45  385153.29
  2004 2019-06-12 07:15:00+00:00                   ...  32337.45  385153.29
  2005 2019-06-12 07:30:00+00:00                   ...  32337.45  385153.29
  2006 2019-06-12 07:45:00+00:00                   ...  32337.45  385153.29
  2007 2019-06-12 08:00:00+00:00                   ...  32337.45  385153.29
  2008 2019-06-12 08:15:00+00:00                   ...  32337.45  385153.29
  2009 2019-06-12 08:30:00+00:00                   ...  32337.45  385153.29
  2010 2019-06-12 08:45:00+00:00                   ...  32337.45  385153.29
  2011 2019-06-12 09:00:00+00:00                   ...  32337.45  385153.29
  2012 2019-06-12 09:15:00+00:00                   ...  32337.45  385153.29
  2013 2019-06-12 09:30:00+00:00                   ...  32337.45  385153.29
  2014 2019-06-12 09:45:00+00:00                   ...  32337.45  385153.29
  2015 2019-06-12 10:00:00+00:00                   ...  32337.45  385153.29
  2016 2019-06-12 10:15:00+00:00                   ...  32337.45  385153.29
  2017 2019-06-12 10:30:00+00:00                   ...  32337.45  385153.29
  2018 2019-06-12 10:45:00+00:00                   ...  32337.45  385153.29
  2019 2019-06-12 11:00:00+00:00                   ...  32337.45  385153.29
  2020 2019-06-12 11:15:00+00:00  

En nu gaan we alles combineren. 
Eerst definieren we een basemap met behulp van een TopoJSON van de `pwo_gebieden`.

Vervolgens maken we een `selection_multi()` object welke reageert op de `stationName`

Daarna maken we het figuur voor de locations en los daarvan het figuur van de series (werken ze ook op zichzelf staand? Probeer maar eens)

En daarna combineren we ze gezamenlijk
De `basemap` + `locations` en deze weer naast elkaar met de `series`

In [33]:
url_topojson = 'https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=pwo_gebieden&contentType=application/json'
data_topojson_remote = altair.topo_feature(url=url_topojson, feature='pwo_gebieden')

basemap = altair.Chart(data_topojson_remote).mark_geoshape(
    fill='lightgray',
    stroke='white'
).encode(tooltip='properties.GEB_NAAM:N')

selector = altair.selection_multi(empty='none', fields=['stationName'], resolve='global')

base = altair.Chart(ts_latlon)

locations = base.mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=altair.Size('mean(value)', title='gemiddelde waterstand (mNAP)'),
    color='stationName',
    tooltip=['stationName', altair.Tooltip('mean(value)', title='gem. waterstand (mNAP)', format='.2f')]
).add_selection(selector
).properties(
    width=250,
    height=250,
    title='overzicht kaart stuwen'
)

series = base.mark_line().encode(
    x='date',
    y=altair.Y('value', title='waterstand binnenstrooms (mNAP)',scale=altair.Scale(zero=False)),
    color='stationName',
    tooltip=['value', 'stationName', altair.Tooltip('date', format='%Y-%m-%d %H:%M')]
).add_selection(selector).transform_filter(
    selector
).properties(
    height=250,
    title='klik in kaart op stuw voor tijdreeks (gebruik shift voor multi)'
)

comb_chart_dynamic = altair.hconcat(
    altair.layer(basemap, locations), 
    series
).configure(
    concat=altair.CompositionConfig(spacing=40),
    legend=altair.LegendConfig(labelLimit=0)
)

comb_chart_dynamic

HConcatChart({
  config: Config({
    concat: CompositionConfig({
      spacing: 40
    }),
    legend: LegendConfig({
      labelLimit: 0
    })
  }),
  hconcat: [LayerChart({
    layer: [Chart({
      data: UrlData({
        format: TopoDataFormat({
          feature: 'pwo_gebieden',
          type: 'topojson'
        }),
        url: 'https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=pwo_gebieden&contentType=application/json'
      }),
      encoding: FacetedEncoding({
        tooltip: Tooltip({
          field: 'properties.GEB_NAAM',
          type: 'nominal'
        })
      }),
      mark: MarkDef({
        fill: 'lightgray',
        stroke: 'white',
        type: 'geoshape'
      })
    }), Chart({
      data:                           date moduleInstanceId  ...         x          y
      0    2019-06-09 13:15:00+00:00                   ...  73695.47  379561.24
      1    2019-06-09 13:30:00+00:00                   ...  73695.47  379561.24
      2    2019-06-09 13:45:00+00:00                   ...  73695.47  379561.24
      3    2019-06-09 14:00:00+00:00                   ...  73695.47  379561.24
      4    2019-06-09 14:15:00+00:00                   ...  73695.47  379561.24
      5    2019-06-09 14:30:00+00:00                   ...  73695.47  379561.24
      6    2019-06-09 14:45:00+00:00                   ...  73695.47  379561.24
      7    2019-06-09 15:00:00+00:00                   ...  73695.47  379561.24
      8    2019-06-09 15:15:00+00:00                   ...  73695.47  379561.24
      9    2019-06-09 15:30:00+00:00                   ...  73695.47  379561.24
      10   2019-06-09 15:45:00+00:00                   ...  73695.47  379561.24
      11   2019-06-09 16:00:00+00:00                   ...  73695.47  379561.24
      12   2019-06-09 16:15:00+00:00                   ...  73695.47  379561.24
      13   2019-06-09 16:30:00+00:00                   ...  73695.47  379561.24
      14   2019-06-09 16:45:00+00:00                   ...  73695.47  379561.24
      15   2019-06-09 17:00:00+00:00                   ...  73695.47  379561.24
      16   2019-06-09 17:15:00+00:00                   ...  73695.47  379561.24
      17   2019-06-09 17:30:00+00:00                   ...  73695.47  379561.24
      18   2019-06-09 17:45:00+00:00                   ...  73695.47  379561.24
      19   2019-06-09 18:00:00+00:00                   ...  73695.47  379561.24
      20   2019-06-09 18:15:00+00:00                   ...  73695.47  379561.24
      21   2019-06-09 18:30:00+00:00                   ...  73695.47  379561.24
      22   2019-06-09 18:45:00+00:00                   ...  73695.47  379561.24
      23   2019-06-09 19:00:00+00:00                   ...  73695.47  379561.24
      24   2019-06-09 19:15:00+00:00                   ...  73695.47  379561.24
      25   2019-06-09 19:30:00+00:00                   ...  73695.47  379561.24
      26   2019-06-09 19:45:00+00:00                   ...  73695.47  379561.24
      27   2019-06-09 20:00:00+00:00                   ...  73695.47  379561.24
      28   2019-06-09 20:15:00+00:00                   ...  73695.47  379561.24
      29   2019-06-09 20:30:00+00:00                   ...  73695.47  379561.24
      ...                        ...              ...  ...       ...        ...
      2000 2019-06-12 06:15:00+00:00                   ...  32337.45  385153.29
      2001 2019-06-12 06:30:00+00:00                   ...  32337.45  385153.29
      2002 2019-06-12 06:45:00+00:00                   ...  32337.45  385153.29
      2003 2019-06-12 07:00:00+00:00                   ...  32337.45  385153.29
      2004 2019-06-12 07:15:00+00:00                   ...  32337.45  385153.29
      2005 2019-06-12 07:30:00+00:00                   ...  32337.45  385153.29
      2006 2019-06-12 07:45:00+00:00                   ...  32337.45  385153.29
      2007 2019-06-12 08:00:00+00:00                   ...  32337.45  385153.29
      2008 2019-06-12 08:15:00+00:00 

Speel met het figuur. Lukt het je om meerdere tijdreeksen in het rechter figuur te krijgen?

Over welke elementen hebben we allemaal een tooltip gedefinieerd?


Laten we ook dit figuur opslaan als HTML pagina binnen de `hkvportal`.

In [0]:
# export visualizatie als een 'in-memory text stream'
export_html = StringIO()
comb_chart_dynamic.save(export_html, format='html')

In [35]:
dp.setEntryDatabase(database='WaterDataScheldestromen', key='kaart_sluizen_series', 
                    data=export_html.getvalue(), 
                    description='kaart sluizen waterstand tijdreeks van 3 stuwen')

available at https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=kaart_sluizen_series&contentType=SET_BY_USER


{'date': '2019-06-12 13:23:49',
 'id': 'b7fb7a77-9b27-4f2a-8e28-0e3f8e61b613',
 'key': 'kaart_sluizen_series'}

Inspectuur het resultaat vanuit de URL in je browser en voeg het toe aan je pagina in SharePoint